In [1]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate
!pip install accelerate
!pip install --user -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from datasets import Dataset
import evaluate
import nltk
import torch

Загружаем отобранные предложения. Предложения отбираются в скрипте precorrect.py

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/My Drive/CorrectDN.json', encoding="utf-8") as f:
    articles = pd.read_json(f, orient='records')
articles.tail()

,text,label
5378,В Москве открылись избирательные участки на вы...,1
5379,Трое детей погибли при обстреле Донецка. ДОНЕЦ...,1
5380,В Липецкой области нейтрализовали два беспилот...,1
5381,Болгария с апреля начнет выдавать россиянам ше...,1
5382,Все участки на выборах президента в Москве отк...,1


Делим на предложения и собираем list с предложениеми

In [5]:
nltk.download('punkt')
sents = []
for ind in articles.index:
  sentArticle = nltk.tokenize.sent_tokenize(articles['text'][ind], language="russian")
  sents += sentArticle
sentences = pd.DataFrame(list(zip(sents)), columns=['text'])
sentences.tail()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,text
49388,"Благодаря тому, что они несут службу, ночь про..."
49389,Избиратели смогут прийти на участок и проголос...
49390,"А граждане, у которых есть полная учетная запи..."
49391,Совет Федерации назначил президентские выборы ...
49392,"Голосование продлится три дня: 15, 16 и 17 марта."


In [6]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds.features

{'text': Value(dtype='string', id=None)}

Делим dataset на train и test (80:20)

In [7]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [8]:
model_checkpoint = "ai-forever/RuM2M100-1.2B"
model = M2M100ForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint, src_lang="ru", tgt_lang="ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/465k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Tokenize dataset

In [9]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], return_tensors="pt")
    result["labels"] = result["input_ids"]
    return result

tokenized_datasets = raw_ds.map(
    tokenize_function, remove_columns=["text"]
)
tokenized_datasets

Map:   0%|          | 0/39514 [00:00<?, ? examples/s]

Map:   0%|          | 0/9879 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 39514
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9879
    })
})

Optimizer

In [10]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [11]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 1
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=default_data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"],
    batch_size=batch_size,
    collate_fn=default_data_collator
)

In [12]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [13]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [14]:
output_dir = '/content/drive/My Drive/fine-train-m2m100'

In [15]:
dataloader_iterator = iter(train_dataloader)
for i in range(1):
  try:
    data = next(dataloader_iterator)
  except StopIteration:
    dataloader_iterator = iter(train_dataloader)
    data = next(dataloader_iterator)
  print (data)
  outputs = model(input_ids=data["input_ids"], \
                  attention_mask=data["attention_mask"], \
                  labels=data["labels"])
  loss = outputs.loss
  print (loss)
#loss.backward()
#optimizer.step()
#lr_scheduler.step()
#optimizer.zero_grad()


{'input_ids': tensor([[[14314,   641,  4126,   512,  1571,  1323,  1993,   186,  6257,   514,
           6603,  1999,  5108,  1778,     5,     2]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]]), 'labels': tensor([[[14314,   641,  4126,   512,  1571,  1323,  1993,   186,  6257,   514,
           6603,  1999,  5108,  1778,     5,     2]]])}


ValueError: too many values to unpack (expected 2)

In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for r in train_dataloader:
        model_inputs = tokenizer(r["text"], return_tensors="pt").to(device)
        # labels = tokenizer(text_target=r["text"], return_tensors="pt").to(device)
        with tokenizer.as_target_tokenizer():
          labels = tokenizer(r["text"], return_tensors="pt").input_ids
        print(model_inputs)
        print(labels)
        outputs = model(**model_inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(batch_size)

    # Evaluation
    model.eval()
    losses = []
    for step, r in enumerate(eval_dataloader):
        with torch.no_grad():
          model_inputs = tokenizer(r["text"], return_tensors="pt").to(device)
          labels = tokenizer(text_target=r["text"], return_tensors="pt").to(device)
          outputs = model(**model_inputs, labels=labels)

        loss = outputs.loss
        losses.append(loss.repeat(1))

    losses = torch.cat(losses)
    losses = losses[: len(raw_ds["test"])]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

  0%|          | 0/39514 [00:00<?, ?it/s]

{'input_ids': tensor([[14314,   567,  7654, 11805,  9890,  1074,    27,  4754,  3765,  1074,
          7765,     5,  3988,   316,  7507,  2096,   292,    39,  1117,  2158,
           749,     4,    39,  4844,   602,  4790,  2324,  5605,  2324,  5985,
           751, 11614,  8252,  5300,  3119,  6779,  1122,  1154,     5,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[14314,   567,  7654, 11805,  9890,  1074,    27,  4754,  3765,  1074,
          7765,     5,  3988,   316,  7507,  2096,   292,    39,  1117,  2158,
           749,     4,    39,  4844,   602,  4790,  2324,  5605,  2324,  5985,
           751, 11614,  8252,  5300,  3119,  6779,  1122,  1154,     5,     2]])
